In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import zipfile

In [ ]:
zf = zipfile.ZipFile('../input/loan-default-prediction/train_v2.csv.zip')
train = pd.read_csv(zf.open('train_v2.csv'), low_memory=False)

In [ ]:
train = train.drop ('id', axis = 1)

In [ ]:
zf1 = zipfile.ZipFile('../input/loan-default-prediction/test_v2.csv.zip')
test = pd.read_csv(zf1.open('test_v2.csv'), low_memory=False)

In [ ]:
test = test.drop ('id', axis = 1)

In [ ]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [ ]:
train = downcast_dtypes(train)
test = downcast_dtypes (test)

In [ ]:
sub = pd.read_csv ('../input/loan-default-prediction/sampleSubmission.csv')

In [ ]:
train.info ()

In [ ]:
train_corr = abs (train.corr ()).fillna (0)

In [ ]:
L = []
idx = []
col = []
for j in train_corr.columns:
    for i in train_corr.index:
        L.append (train_corr.loc [i,j])
        idx.append (i)
        col.append (j)
        
idx_df = pd.Series (idx)
col_df = pd.Series (col)
L_df = pd.Series (L)

corr = pd.concat ([idx_df, col_df, L_df], axis = 1)
corr.columns = ['idx', 'col', 'corr']

corr_plus = corr.loc [(corr ['corr'] > 0.6) & (corr ['corr'] < 1), :]
corr_plus = corr_plus.drop_duplicates (subset = ['corr'], keep = 'first')

In [ ]:
corr_plus ['idx'].value_counts ().head (300)

In [ ]:
to_drop = list (corr_plus ['idx'].value_counts ().head (300).index)

In [ ]:
train = train.drop (to_drop, axis = 1)
test = test.drop (to_drop, axis = 1)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3,shuffle=True, random_state=None)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error as mae
from sklearn.pipeline import Pipeline

pipe = Pipeline ([('scaler', StandardScaler ()), ('pca',PCA (n_components = 100))])

In [ ]:
MAE = []
for itrain, ival in kf.split (train):
    train_sub, val_sub = train.iloc [itrain, :], train.iloc [ival, :]
    
    train_sub_num = train_sub.loc[:,(train_sub.dtypes == 'float32')|(train_sub.dtypes == 'int16')]
    val_sub_num = val_sub.loc[:,(val_sub.dtypes == 'float32')|(val_sub.dtypes == 'int16')]
    train_sub_cat = train_sub.loc [:, (train_sub.dtypes == 'object')]
    val_sub_cat = val_sub.loc [:, (val_sub.dtypes == 'object')]
    
    train_sub_num.loc [:,train_sub_num.dtypes == 'float32'] = train_sub_num.loc [:,train_sub_num.dtypes == 'float32'].apply (lambda x : x.fillna (x.mean ()).astype ('float32'))
    train_sub_num.loc [:,train_sub_num.dtypes == 'int16'] = train_sub_num.loc [:,train_sub_num.dtypes == 'int16'].apply (lambda x : x.fillna (x.mean ()).astype ('int16'))
    val_sub_num.loc [:,val_sub_num.dtypes == 'float32'] = val_sub_num.loc [:,val_sub_num.dtypes == 'float32'].apply (lambda x : x.fillna (x.mean ()).astype ('float32'))
    val_sub_num.loc [:,val_sub_num.dtypes == 'int16'] = val_sub_num.loc [:,val_sub_num.dtypes == 'int16'].apply (lambda x : x.fillna (x.mean ()).astype ('int16'))
    
    train_sub_cat = train_sub_cat.apply (lambda x : x.fillna (x.value_counts ().idxmax))
    
    
        
    val_sub_cat = val_sub_cat.apply (lambda x : x.fillna (x.value_counts ().idxmax))
    
    
    
    y_train = train_sub_num ['loss']
    X_train = train_sub_num.drop ('loss', axis = 1)
    y_val = val_sub_num ['loss']
    X_val = val_sub_num.drop ('loss', axis = 1)
    
    X_train_norm = pipe.fit_transform (X_train)
    X_val_norm = pipe.fit_transform (X_val)
    
    rf = RandomForestClassifier(n_estimators= 40, max_depth = 12)
    rf.fit (X_train_norm, y_train)
    y_lr = rf.predict (X_val_norm)
    
    MAE.append (mae (y_lr, y_val)) 
    print (MAE)    

In [ ]:
test.info ()

In [ ]:
test_num = test.loc[:,(test.dtypes == 'float32')|(test.dtypes == 'int16')]
test_num.loc [:,test_num.dtypes == 'float32'] = test_num.loc [:,test_num.dtypes == 'float32'].apply (lambda x : x.fillna (x.mean ()).astype ('float32'))
test_num.loc [:,test_num.dtypes == 'int16'] = test_num.loc [:,test_num.dtypes == 'int16'].apply (lambda x : x.fillna (x.mean ()).astype ('int16'))

In [ ]:
test_num.isnull ().sum ().sum ()

In [ ]:
X_test = pipe.fit_transform (test_num)
y_pred = rf.predict (X_test)
sub ['loss'] = pd.Series (y_pred)

In [ ]:
sub.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")